In [1]:
import pandas as pd
import numpy as np

In [3]:
df_ratings = pd.read_json('data/ratings.jsonl', lines=True)
df_ratings.head()

,UserId,ItemId,Timestamp,Rating
0,c4ca4238a0,91766eac45,2013-10-05 22:00:50,8
1,c81e728d9d,5c739554f7,2013-08-17 16:26:38,9
2,c81e728d9d,48f6d7ce7c,2013-08-17 13:28:27,8
3,c81e728d9d,e9318d627a,2013-06-15 15:38:09,1
4,a87ff679a2,17e6357973,2014-01-31 23:27:59,8


In [7]:
df_predictions = pd.read_csv('target_predictions.csv')
df_predictions.head()

,UserId,ItemId,Rating
0,0006246bee,01d2404d4c,6.899188
1,0006246bee,03d43fdf92,6.511836
2,0006246bee,0808a9666b,7.196155
3,0006246bee,0a5d7dd6f6,7.212797
4,0006246bee,0bab4a8104,6.579674
